<a href="https://colab.research.google.com/github/TomPhanAnh/Kaggle-Solution/blob/main/DaZone_Round2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# **ZALOPAY INTRODUCTION**

ZaloPay is an **E-wallet** established at the end of 2016 and is
part of Vietnamese unicorn VNG’s digital ecosystem, which
also includes gaming, media and cloud computing.

In January 2020, ZaloPay launched on Zalo platform, with the
aim to become the **“Vietnamese citizens’ E-wallet”.** ZaloPay
provides daily usage in-app services such as money transfer,
phone top-up, and paying bills. With a ready presence on local
users’ smartphones and robust product offerings, ZaloPay has
competitive advantages to become the champion e-wallet.

In May 2022, ZaloPay reached 10M active users, which was a
significant milestone. After a few years of traversing the
digital payment market, we have helped to foster a cashless
society by creating convenient payment methods for our
consumers' daily lives.

According to Circular 23/2019/TT-NHNN, each individual must accurately fill
out their personal information to validate their account and minimize risks
while making a payment. Before making payments for services, e-wallets must
be linked to the owner's bank account or debit card. As a result, ZaloPay has
always **prioritized the growth of associated banks and studying and optimizing
the flow of connecting banks to provide all clients with the most convenient
wallet.**

## **Context**

There are two systems of interest here: **front-end** and **back-end** systems. The **front-end system tracks **users’ interaction with the app (users stayed on
which screen, clicked on which button, at which time, etc.).The back-end
system** tracks the request in the back end that is created as a result of certain actions on the front-end. For example, when users clicked on the button “Mua ngay (buying immediately)” on an item, there’s a request on the backend at the same time and its status reflects whether users purchased successfully or not.

**For the front-end dataset:** Each screen in the product flow that you will
investigate in this assignment has its ID. The at-issues screens are numbered
from 11 to 18, where Screen 11 is the beginning of the flow where users will fill out their bank account info and click “Lien ket ngay (link immediately)” (see the illustrative at the end of the assignment). Screens 12 to 18 reflect different meanings (e.g. the outcomes of the “Lien ket ngay” action, and our
corresponding features for various outcomes).

**For the back-end dataset:** when users click “Liên kết ngay” on screenID 11,
a request will be recorded in the backend.csv. There are a lot of statuses here, but the one of interest is “-FACJ” status and “B”. When users encounter the “-FACJ” error, they will interact with another flow.

We are testing two UIs for users who encounter “-FACJ” error. Let’s call these
“new UI” and “old UI”. Only a subset of users can see the new UI, which includes
screenID 12, 13, 14, 15, 16, 17, 18. The old UI does not have an associated screenID, so it temporarily assumes users that have “-FACJ” error but don’t see screenID 12 through 18 will see the old UI.

The old UI and new UI are expected to help users retry and, hopefully, succeed.
Users that finish interacting with the flow after “-FACJ” will come back to
screenID 11 and submit another request to backend.csv. A successful request has
a “B” status.

**Preprocessing dataset**

In [ ]:
front_end = pd.read_csv("Front-end dataset.csv",delimiter = " ")
front_end = front_end['eventid\tid\tscreenid\ttimestamp\t\t\t,'].str.split("\t",expand = True)
front_end.drop([4,5,6], axis = 1, inplace = True)
front_end.columns = ["eventid","id","screenid","timestamp"]



In [ ]:
def remove_comma(x):
  if "'" in x:
    return x.replace("'"," ").strip()
  else:
    return x
front_end["id"] = front_end["id"].apply(lambda x: remove_comma(x))
front_end["screenid"] = front_end["screenid"].apply(lambda x: remove_comma(x))
front_end["timestamp"] = front_end["timestamp"].apply(lambda x: remove_comma(x))

In [ ]:
front_end["timestamp"] = pd.to_datetime(front_end["timestamp"],unit = "ms")

In [ ]:
front_end.drop_duplicates(inplace = True)

## **Backend Dataset**

In [ ]:
back_end = pd.read_csv("Back-end dataset.csv")
back_end = back_end['timestamp\tstepresult\tid'].str.split("\t",expand = True)
back_end.rename(columns = {0:"timestamp",1:"stepresult",2:"id"}, inplace =True)
back_end["timestamp"] = pd.to_datetime(back_end['timestamp'], unit = 'ms')

## **Merging Dataset**

In [ ]:
count_front = front_end.groupby(["id","screenid","eventid"]).agg({"eventid":"count"}).reset_index(level = [0,1])
max_front = count_front.groupby(["id","screenid"]).agg({"eventid":"max"}).reset_index().sort_values(["id","screenid"])

In [ ]:
dataset = pd.DataFrame(max_front["id"].values, columns = ["id"])
dataset["screen11"] = 0
dataset["screen1218"] = 0

In [ ]:
count_front.rename(columns = {"eventid":"count"},inplace = True)
count_front = count_front.reset_index()

In [ ]:
merge_data = count_front.merge(front_end,on = ["eventid","id"], how = "right")
merge_data["screenid_x"] = merge_data["screenid_x"].astype("int64")

In [ ]:
# Create the dataset for screenid == 11
merge_data11 = merge_data.loc[merge_data["screenid_x"] == 11]
new_data = pd.DataFrame(columns = merge_data.columns)
for i in merge_data11["id"].value_counts().index:
  
  # Test the condition to take the most interactive eventid of old UI
  count = max(merge_data11.loc[(merge_data11["id"] == i)]["count"])
  eventid = merge_data11.loc[(merge_data11["id"] == i ) & 
                           (merge_data11["count"] == count)].sort_values(["timestamp","eventid"])["eventid"].value_counts().index[0]

  # Concatenate the Data with condition
  df = merge_data11.loc[(merge_data11["id"] == i) & 
                 (merge_data11["eventid"] == eventid)]  
            
  new_data = pd.concat([df,new_data])

In [ ]:
# Create the dataset for screenid != 11
merge_data1218 = merge_data.loc[merge_data["screenid_x"] != 11]
new_data1218 = pd.DataFrame(columns = merge_data.columns)
for i in merge_data1218["id"].value_counts().index:

  # Test the condition to take the most interactive eventid of new UI
  count = max(merge_data1218.loc[(merge_data1218["id"] == i)]["count"])
  eventid = merge_data1218.loc[(merge_data1218["id"] == i ) & 
                           (merge_data1218["count"] == count)].sort_values(["timestamp","eventid"])["eventid"].value_counts().index[0]

  # Concatenate the Data with condition
  df = merge_data1218.loc[(merge_data1218["id"] == i) & 
                 (merge_data1218["eventid"] == eventid)]  
            
  new_data1218 = pd.concat([df,new_data1218])

In [ ]:
# Concate to get timestamp for each loop
data_Q1 = pd.concat([new_data,new_data1218])

# **Task 1:Prepare a report that answers the following questions**

**Q1. What is the retry rate for each UI? Does the new UI improve the retry
rate?**

In [ ]:
# What is the retry rate for each UI? 
# 1 is new
# 0 is old
Q1 = data_Q1.drop_duplicates(["id","screenid_x"])

# create the dataframe to check UI old or new
question1 = pd.DataFrame(data_Q1["id"].value_counts().index, columns = ["id"])
question1["screen11"] = 0
question1["screen1218"] = 0

In [ ]:
# Define UI
question1["UI"] = question1.apply(lambda x: 1 if x["screen1218"] > 0 else 0, axis = 1)
question1["screen11"] = question1["screen11"].astype("int64")
question1["screen1218"] = question1["screen1218"].astype("int64")

# Calculate retry rate for each UI
# In case New UI enter in screen12 to screen 18 more than log-in (screen11) => no retry rate = 0
# Else New UI enter in screen12 to screen 18 less than log-in (screen11)    => retry rate is available
question1["retry_times"] = question1.apply(lambda x: max(0,x["screen11"] - x["screen1218"]), axis = 1)

# retry rate of new_UI
new_iu_retry = sum(question1.loc[question1["UI"] == 1]['retry_times'])/(
              
                sum(question1.loc[question1["UI"] == 1]['retry_times'])
              + sum(question1.loc[question1["UI"] == 1]["screen1218"]))

# Suppose the last time of old Ui does mean they stop => the last eventid doesn't mean a retry but a stop => don't include in the number of retry

# retry rate of old_UI
old_iu_retry = (sum(question1.loc[question1["UI"] == 0]['screen11']) -
                len(question1.loc[question1["UI"] == 0]['screen11']))/sum(question1.loc[question1["UI"] == 0]['screen11'])

In [ ]:
print("The retry rate of New UI: {}".format(round(new_iu_retry,2)))
print("The retry rate of Old UI: {}".format(round(old_iu_retry,2)))

**Q1.2: Does the new UI improve the retry rate?**

In [ ]:
# Create the hour interval for frontend dataset
front_end["Year"]  = front_end["timestamp"].apply(lambda x: x.year)
front_end["Month"] = front_end["timestamp"].apply(lambda x: x.month)
front_end["Day"]   = front_end["timestamp"].apply(lambda x: x.day)
front_end["Hour"]  = front_end["timestamp"].apply(lambda x: x.hour)
front_end["timestamp2"] = front_end.apply(lambda x: pd.datetime(x["Year"],x["Month"],x["Day"],x["Hour"]), axis = 1)

In [ ]:
Q12 = front_end.merge(question1[["id","UI"]], on = "id", how = "right")
Q12.sort_values(["id","timestamp"])

In [ ]:
Q12.loc[(Q12["screenid"] == "11") 
                   & (Q12["UI"] == 1)].groupby(
                  by = ["timestamp2"]).count().reset_index()[["timestamp2","eventid"]].rename(columns = {"eventid":"screen11"})

In [ ]:
Q12_noname = pd.DataFrame(columns = Q12.columns)
for i in Q12.loc[(Q12["screenid"] != "11") & (Q12["UI"] == 1)]["timestamp2"].unique():
  df12 = Q12.loc[(Q12["screenid"] != "11") 
                   & (Q12["UI"] == 1) & (Q12["timestamp2"] == i)].groupby(by = ["timestamp2","screenid"]).count().reset_index()
  a = max(xx.loc[(df12["timestamp2"] == i)]["eventid"])






In [ ]:
Q12.loc[(Q12["screenid"] != "11") 
                   & (Q12["UI"] == 1) & (Q12["timestamp2"] == "2021-03-15 08:00:00")].groupby(by = ["timestamp2","screenid"]).count().reset_index()

In [ ]:
xx = Q12.loc[(Q12["screenid"] != "11") 
                   & (Q12["UI"] == 1)].groupby(by = ["timestamp2","screenid"]).count().reset_index()
xx.head()

In [ ]:
max(Q12.loc[(Q12["screenid"] != "11") 
                   & (Q12["UI"] == 1)].groupby(by = ["timestamp2","screenid"]).count().reset_index()["eventid"])

In [ ]:
a = max(xx.loc[(xx["timestamp2"] == "2021-03-15 08:00:00")]["eventid"])
a

In [ ]:
xx.loc[(xx["timestamp2"] == "2021-03-15 08:00:00") & (xx["eventid"] == a)]

In [ ]:
Q12_timestamp = Q12_1_sc11.merge(Q12_1_sc12, on = "timestamp2", how = "left").fillna(0)
Q12_timestamp["retry_times"] = Q12_timestamp.apply(lambda x: max(0,x["screen11"] - x["screen1218"]), axis = 1)

In [ ]:
Q12_timestamp["retry_rate"] = Q12_timestamp.apply(lambda x: x["retry_times"]/ (x["retry_times"]+ x["screen1218"]), axis = 1)

In [ ]:
fig = plt.figure(figsize = (15,8))
aa = Q12_timestamp[["timestamp2","retry_rate"]]
plt.plot(aa["timestamp2"],aa["retry_rate"])